### Задача 1. Сколько выбросов удалять
  

С одной стороны, при удалении выбросов снижается дисперсия, что приводит к увеличению чувствительности теста. С другой стороны, при удалении выбросов уменьшается размер выборки, что приводит к уменьшению чувствительности.

Сравните мощности тестов с разной долей удаляемых данных. Используйте данные о времени работы бэкенда `2022-04-01T12_df_web_logs.csv` в период с 2022-03-01 по 2022-03-08. Уровень значимости — 0.05. Размеры групп — 1000 человек (размер выборок будет больше, так как на одного человека приходится много значений). Проверяем гипотезу о равенстве средних с помощью теста Стьюдента. Ожидаемый эффект — увеличение времени обработки на 1%. Эффект в синтетических А/В-тестах добавляем умножением на константу.

В ответ введите номера вариантов, упорядоченные по уменьшению мощности. Например, «12345» означает, что вариант 1 обладает наибольшей мощностью, а вариант 5 — наименьшей.

1. Удалить 0.02% выбросов;

2. Удалить 0.2% выбросов;

3. Удалить 2% выбросов;

4. Удалить 10% выбросов;

5. Удалить 20% выбросов.

Удалить 2% выбросов означает, что нужно убрать по 1% минимальных и максимальных значений выборки. То есть оставить значения, которые лежат между `np.quantile(values, 0.01)` и `np.quantile(values, 0.99)`. Квантили вычислять для каждой групп отдельно.

In [1]:
import pandas as pd
from datetime import datetime

import numpy as np
from scipy.stats import ttest_ind

In [2]:
df_raw = pd.read_csv('./2022-04-01T12_df_web_logs.csv')

In [3]:
df_raw.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [4]:
df_raw['date'] = pd.to_datetime(df_raw['date'])
df = df_raw[(df_raw['date']>=datetime(2022, 3, 1))&(df_raw['date']<datetime(2022, 3, 8))].copy(deep=True)
users = df['user_id'].unique()

In [5]:
alpha = 0.05
sample_size = 1000
effect = 0.01

In [6]:
quantiles = {
    '1':0.0001, #0.02%
    '2':0.001,  #0.2%
    '3':0.01,   #2%
    '4':0.05,   #10%
    '5':0.1     #20%
}

quantile2errors = {q: [] for q in quantiles.values()}

In [7]:
for _ in range(1000):
    a_users, b_users = np.random.choice(users, (2, sample_size), False)
    a_values = df.loc[df['user_id'].isin(a_users), 'load_time'].values
    b_values = df.loc[df['user_id'].isin(b_users), 'load_time'].values * (1 + effect)

    for q in quantiles.values():
        a_values_filt = a_values[(a_values > np.quantile(a_values, q))&(a_values < np.quantile(a_values, 1 - q))]
        b_values_filt = b_values[(b_values > np.quantile(b_values, q))&(b_values < np.quantile(b_values, 1 - q))]

        pvalue = ttest_ind(a_values_filt, b_values_filt).pvalue
        quantile2errors[q].append(pvalue > alpha)


In [8]:
def process_res(quantile2errors):
    '''Обработка результатов, подсчет мощности'''
    data = [(idx+1, quantile, np.mean(errors), errors)
            for idx, (quantile, errors) in enumerate(quantile2errors.items())]
    #сортировка по доле ошибок
    data.sort(key=lambda x: x[2])

    #проверка, что доля ошибок значимо отличается
    for i in range(1, len(data)):
        pvalue = ttest_ind(data[i][3], data[i-1][3]).pvalue
        if pvalue < 0.05:
            msg = f'pvalue = {pvalue:0.04f}, оценка мощности значимо отличается от предыдущей'
        else:
            msg = f'pvalue = {pvalue:0.04f}, оценка мощности значимо не отличается от предыдущей'
        print(f'№ answ = {data[i][0]}, quantile = {data[i][1]}, power = {1-data[i][2]:0.3f}, {msg}')
    print('answ: ', ''.join([str(x[0]) for x in data]))

process_res(quantile2errors)

№ answ = 4, quantile = 0.05, power = 0.959, pvalue = 0.2302, оценка мощности значимо не отличается от предыдущей
№ answ = 3, quantile = 0.01, power = 0.937, pvalue = 0.0267, оценка мощности значимо отличается от предыдущей
№ answ = 2, quantile = 0.001, power = 0.356, pvalue = 0.0000, оценка мощности значимо отличается от предыдущей
№ answ = 1, quantile = 0.0001, power = 0.102, pvalue = 0.0000, оценка мощности значимо отличается от предыдущей
answ:  54321


### Задача 2. Сколько выбросов удалять — 2
  

Выполните то же задание, изменив способ добавления эффекта. Эффект в синтетических А/В-тестах добавляем добавлением константы к 1% данных.

В ответ введите номера вариантов упорядоченные по уменьшению мощности. Например, «12345» означает, что вариант 1 обладает наибольшей мощностью, а вариант 5 — наименьшей.

1. Удалить 0.02% выбросов;

2. Удалить 0.2% выбросов;

3. Удалить 2% выбросов;

4. Удалить 10% выбросов;

5. Удалить 20% выбросов.

Удалить 2% выбросов означает, что нужно убрать по 1% минимальных и максимальных значений выборки. То есть оставить значения, которые лежат между np.quantile(values, 0.01) и np.quantile(values, 0.99). Квантили вычислять для каждой группы отдельно.

In [9]:
df_raw = pd.read_csv('./2022-04-01T12_df_web_logs.csv')

df_raw['date'] = pd.to_datetime(df_raw['date'])
df = df_raw[(df_raw['date']>=datetime(2022, 3, 1))&(df_raw['date']<datetime(2022, 3, 8))].copy(deep=True)
users = df['user_id'].unique()

In [10]:
alpha = 0.05
sample_size = 1000
effect = 0.01

In [11]:
quantiles = {
    '1':0.0001, #0.02%
    '2':0.001,  #0.2%
    '3':0.01,   #2%
    '4':0.05,   #10%
    '5':0.1     #20%
}

quantile2errors = {q: [] for q in quantiles.values()}

In [12]:
for _ in range(2000):
    a_users, b_users = np.random.choice(users, (2, sample_size), False)
    a_values = df.loc[df['user_id'].isin(a_users), 'load_time'].values
    b_values = df.loc[df['user_id'].isin(b_users), 'load_time'].values.copy()

    # Применяем эффект только к 1% случайно выбранных значений
    mean_val = b_values.mean()
    n = int(0.01 * len(b_values))
    idx = np.random.choice(len(b_values), n, replace=False)
    add_value = effect * mean_val * len(b_values) / len(idx)
    mask = np.zeros(len(b_values))
    mask[idx] += 1
    b_values = b_values + mask * add_value

    for q in quantiles.values():
        a_values_filt = a_values[(a_values > np.quantile(a_values, q)) & (a_values < np.quantile(a_values, 1 - q))]
        b_values_filt = b_values[(b_values > np.quantile(b_values, q)) & (b_values < np.quantile(b_values, 1 - q))]

        pvalue = ttest_ind(a_values_filt, b_values_filt).pvalue
        quantile2errors[q].append(pvalue > alpha)


📌 Задача кода

Добавить эффект (effect, например 0.01 или 0.02) только к 1% случайных значений массива b_values, но так, чтобы среднее значение всего массива увеличилось на effect * mean_val (т.е. в среднем на 1% или 2%).

🔍 Пошаговый разбор

mean_val = b_values.mean()

Считаем среднее значение до изменения — это понадобится, чтобы задать изменение в относительных единицах (effect * mean_val).

n = int(0.01 * len(b_values))
Определяем количество значений, к которым мы хотим добавить эффект — это 1% от длины массива b_values.

idx = np.random.choice(len(b_values), n, replace=False)
Выбираем n случайных индексов в массиве b_values, без повторений, к которым мы и применим эффект.

add_value = effect * mean_val * len(b_values) / len(idx)
Вот здесь самая интересная и нетривиальная часть.

Цель: добиться среднего прироста по всему массиву b_values равного effect * mean_val, но изменяя только n значений.

Поэтому:

Сколько должно добавиться всего по массиву? → effect * mean_val * len(b_values)

Сколько значений мы изменяем? → len(idx) или n


Это значит: добавив add_value к 1% значений, весь массив b_values сдвинется вверх на effect * mean_val в среднем.

mask = np.zeros(len(b_values))
Создаём массив из нулей той же длины, что и b_values.

mask[idx] += 1
В тех местах, которые мы выбрали ранее (idx), ставим 1, остальное остаётся 0.

Таким образом, mask — это индикатор того, к каким индексам нужно применить add_value.

b_values = b_values + mask * add_value
Добавляем add_value только тем элементам массива, которые указаны в mask.

Элементы с mask = 0 останутся без изменений, а элементы с mask = 1 получат +add_value.

💡 Итог

Ты:

изменяешь только 1% случайных значений,

но увеличиваешь среднее значение всего массива b_values на effect * mean_val, что эквивалентно 1% увеличению, если effect = 0.01.

In [13]:
def process_res(quantile2errors):
    '''Обработка результатов, подсчет мощности'''
    data = [(idx+1, quantile, np.mean(errors), errors)
            for idx, (quantile, errors) in enumerate(quantile2errors.items())]
    #сортировка по доле ошибок
    data.sort(key=lambda x: x[2])

    #проверка, что доля ошибок значимо отличается
    for i in range(1, len(data)):
        pvalue = ttest_ind(data[i][3], data[i-1][3]).pvalue
        if pvalue < 0.05:
            msg = f'pvalue = {pvalue:0.04f}, оценка мощности значимо отличается от предыдущей'
        else:
            msg = f'pvalue = {pvalue:0.04f}, оценка мощности значимо не отличается от предыдущей'
        print(f'№ answ = {data[i][0]}, quantile = {data[i][1]}, power = {1-data[i][2]:0.3f}, {msg}')
    print('answ: ', ''.join([str(x[0]) for x in data]))

process_res(quantile2errors)

№ answ = 2, quantile = 0.001, power = 0.355, pvalue = 0.0000, оценка мощности значимо отличается от предыдущей
№ answ = 5, quantile = 0.1, power = 0.326, pvalue = 0.0572, оценка мощности значимо не отличается от предыдущей
№ answ = 4, quantile = 0.05, power = 0.298, pvalue = 0.0606, оценка мощности значимо не отличается от предыдущей
№ answ = 1, quantile = 0.0001, power = 0.100, pvalue = 0.0000, оценка мощности значимо отличается от предыдущей
answ:  32541


### Задача 3. Функция удаления выбросов

Реализуйте функцию process_outliers.

Шаблон решения


```python
import pandas as pd


def process_outliers(metrics, bounds, outlier_process_type):
    """Возвращает новый датафрейм с обработанными выбросами в измерениях метрики.

    :param metrics (pd.DataFrame): таблица со значениями метрики
        со столбцами ['user_id', 'metric'].
    :param bounds (tuple[float, float]): нижняя и верхняя границы метрики. Всё что
        не попало между ними считаем выбросами.
    :param outlier_process_type (str): способ обработки выбросов. Возможные варианты:
        'drop' - удаляем измерение,
        'clip' - заменяем выброс на значение ближайшей границы (lower_bound, upper_bound).
    :return df: таблица со столбцами ['user_id', 'metric']
    """
        # YOUR_CODE_HERE

```

Пример

```python
metrics = pd.DataFrame({'user_id': [1, 2, 3], 'metric': [1., 2, 3]})
bounds = (0.1, 2.2,)
outlier_process_type = 'drop'
result = process_outliers(metrics, bounds, outlier_process_type)
# result = pd.DataFrame({'user_id': [1, 2], 'metric': [1.0, 2.0]})

outlier_process_type = 'clip'
result = process_outliers(metrics, bounds, outlier_process_type)
# result = pd.DataFrame({'user_id': [1, 2, 3], 'metric': [1.0, 2.0, 2.2]})




In [27]:
import pandas as pd


def process_outliers(metrics, bounds, outlier_process_type):
    """Возвращает новый датафрейм с обработанными выбросами в измерениях метрики.

    :param metrics (pd.DataFrame): таблица со значениями метрики
        со столбцами ['user_id', 'metric'].
    :param bounds (tuple[float, float]): нижняя и верхняя границы метрики. Всё что
        не попало между ними считаем выбросами.
    :param outlier_process_type (str): способ обработки выбросов. Возможные варианты:
        'drop' - удаляем измерение,
        'clip' - заменяем выброс на значение ближайшей границы (lower_bound, upper_bound).
    :return df: таблица со столбцами ['user_id', 'metric']
    """
    lower_bound = bounds[0]
    upper_bound = bounds[1]

    if outlier_process_type == 'drop':
        metrics = metrics.loc[(metrics['metric'] >= lower_bound)&(metrics['metric'] <= upper_bound)]
    elif outlier_process_type == 'clip':
        metrics.loc[(metrics['metric'] >= upper_bound), 'metric'] = upper_bound
        metrics.loc[(metrics['metric'] <= lower_bound), 'metric'] = lower_bound
    else:
        raise ValueError('Неверное значение outlier_process_type')
    return metrics


In [28]:
#тест работы функции
metrics = pd.DataFrame({'user_id': [1, 2, 3], 'metric': [1., 2, 3]})
bounds = (0.1, 2.2,)
outlier_process_type = 'drop'
result = process_outliers(metrics, bounds, outlier_process_type)
print(result)
# result = pd.DataFrame({'user_id': [1, 2], 'metric': [1.0, 2.0]})

   user_id  metric
0        1     1.0
1        2     2.0


In [29]:
outlier_process_type = 'clip'
result = process_outliers(metrics, bounds, outlier_process_type)
print(result)
# result = pd.DataFrame({'user_id': [1, 2, 3], 'metric': [1.0, 2.0, 2.2]})

   user_id  metric
0        1     1.0
1        2     2.0
2        3     2.2
